In [54]:
def find_n_smallest(data,k,d):
    from operator import itemgetter
    import pandas as pd
    dic={i:n for i,n in enumerate(data[k])}
    doc=sorted(dic.items(),key=itemgetter(1))[1:(d+1)]
    key=[a[0] for a in doc]
    per=[a[1] for a in doc]
    df_doc_similar=pd.DataFrame({'key':key,
                                'per':per})
    return df_doc_similar

def find_n_largest(data,k,d):
    from operator import itemgetter
    import pandas as pd
    dic={i:n for i,n in enumerate(data[k])}
    doc=sorted(dic.items(),key=itemgetter(1),reverse=True)[0:d]
    key=[a[0] for a in doc]
    per=[a[1] for a in doc]
    df_doc_largest=pd.DataFrame({'key':key,
                                'per':per})
    return df_doc_largest

def similar_song_id(func,song,data,d):
    import pandas as pd
    s_song=[]
    for i in range(0, len(data)):
        a=func(data,i,d)
        for j in range(0,len(a)):
            s_song.append(song['songid'][a['key'][j]])
    new_df_3=pd.DataFrame(columns=range(0,d))
    for i in range(0,len(s_song),d):
        for j in range(0,d):
            new_df_3.loc[i,j]=s_song[i+j]
    colnames=['Sim'+str(a) for a in range(1,d+1)]
    new_df_3.columns=colnames
    return new_df_3


def similar_song_singer_title(func,song,data,d):
    import pandas as pd
    s_song=[]
    for i in range(0, len(data)):
        a=func(data,i,d)
        for j in range(0,len(a)):
            s_song.append(song['songtitle'][a['key'][j]]+'/'+song['singer'][a['key'][j]])
    new_df_3=pd.DataFrame(columns=range(0,d))
    for i in range(0,len(s_song),d):
        for j in range(0,d):
            new_df_3.loc[i,j]=s_song[i+j]
    colnames=['Sim'+str(a) for a in range(1,d+1)]
    new_df_3.columns=colnames
    return new_df_3


def similar_doc(func,data,d):
    import pandas as pd
    s_sim=[]
    for i in range(0,len(data)):
        a=func(data,i,d)
        for j in range(0,len(a)):
            s_sim.append(str(a['key'][j])+'/'+str(a['per'][j]))
    new_df_3=pd.DataFrame(columns=range(0,d))
    for i in range(0,len(s_sim),d):
        for j in range(0,d):
            new_df_3.loc[i,j]=s_sim[i+j]
    colnames=['Sim'+str(a) for a in range(1,d+1)]
    new_df_3.columns=colnames
    return new_df_3

def get_cosine_sim(data,n,song):
    import numpy as np
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity
    sim_song_df=pd.DataFrame(columns=song.columns)
    for i in range(0,len(data.columns)-1):
        df=song.loc[song['songid']==data.loc[n][i]]
        sim_song_df=pd.concat([sim_song_df,df])
    song_index = sim_song_df[['songid']] 
    song_index = song_index.songid.tolist() # Sim1,2,3 의 songid
    tag_mean = np.array(sim_song_df.drop(['songid','title','artist'], axis=1).mean(axis=0))#.astype(np.float64)) # Sim1,2,3 노래 태그 평균
    tag_mean = tag_mean.reshape(1,-1)
    sim = cosine_similarity(tag_mean,np.array(song.iloc[:,3:]))
    sim = sim.flatten().tolist()
    recom_df = pd.DataFrame({'songid' : song.songid.tolist(),
                             'songtitle': song.title.tolist(),
                             'singer' : song.artist.tolist(),
                             'sim' : sim})
    #final_recom = recom_df.sort_values('sim', ascending= False).head(1)
    final_recom = recom_df.merge(like,on='songid').sort_values('sim', ascending= False)
    final_recom = final_recom[final_recom.like>final_recom['like'].mean()].head(1)
    story_songtag=song_df[song_df.songid==data.storysong.values[n]]
    recom_songtag=song_df[song_df.songid==final_recom.songid.values[0]]
    cs=cosine_similarity(story_songtag.iloc[:,3:],recom_songtag.iloc[:,3:])
    final_df=pd.DataFrame({'story':story_songtag['title'].values+'/'+story_songtag['artist'].values,
                          'recom':recom_songtag['title'].values+'/'+recom_songtag['artist'].values,
                          'sim':cs.flatten()})
    return final_df

def get_tag_mean(data,n,song):
    import numpy as np
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity
    sim_song_df=pd.DataFrame(columns=song.columns)
    for i in range(0,len(data.columns)-1):
        df=song.loc[song['songid']==data.loc[n][i]]
        sim_song_df=pd.concat([sim_song_df,df])
    tag_mean = np.array(sim_song_df.drop(['songid','title','artist'], axis=1).mean(axis=0))#.astype(np.float64)) # Sim1,2,3 노래 태그 평균
    tag_mean = tag_mean.reshape(1,-1)
    return tag_mean

story=pd.read_excel('/home/kookmin/work/데이터저장/only_text_180606_12pm.xlsx')
story2=story.merge(song_df2[['songid','title','artist']],right_on=['title','artist'],left_on=['songtitle','singer'])
for i in range(0, len(story2)):
    if len(story2['text'][i])<=500:
        story2['text'][i]=story2['text'][i]*2
    else:
        next

sentences=list(story2['text'])
from konlpy.tag import Twitter
tok=Twitter()
textpos=[]
for elm in sentences:
    pos=[]
    for tk in tok.pos(elm,norm=False,stem=False):
        if len(tk[0])>1:
            if tk[1] in ['Noun','Adjective','KoreanParticle']:#,'Verb','Adverb','Conjunction','Determiner','Exclamation','VerbPrefix']:
                if tk[1]!="Noun":
                    pos.append(tk[0][:2])
                else:
                    pos.append(tk[0])
    textpos.append(' '.join(pos))
story2['textpos']=textpos

# 데이터 부르기

In [194]:
import pickle
with open('story_textpos_0522.pkl','rb') as f:
    story=pickle.load(f)
story=story.drop([1155,1159,902,765,767,528,529,530],0)
story.index=range(0,len(story))
songtitle=story[['songid','songtitle','singer']]

In [125]:
import pandas as pd
like=pd.read_excel('/home/kookmin/work/데이터저장/song+like.xlsx')

import pandas as pd
import pickle
with open('song_df2.pkl','rb') as f:
    song_df2=pickle.load(f)
song_df2=pd.concat([song_df2.iloc[:,[0,-1,-2]],song_df2.iloc[:,1:-2]],axis=1)

In [4]:
import pickle
with open('song_df2.pkl','rb') as f:
    song_df3=pickle.load(f)

In [195]:
import pickle
with open('song_df2_rate.pkl','rb') as f:
    song_df2=pickle.load(f)
song_df2=pd.concat([song_df2.iloc[:,[0,-1,-2]],song_df2.iloc[:,1:-2]],axis=1)

In [127]:
song_df2.columns

Index(['songid', 'artist', 'title', '가게', '가까이', '가깝', '가난', '가내', '가도', '가두',
       ...
       '흔하', '흥분', '희망', '힐링', '힘겹', '힘든', '힘들', '힘차', '힙스터', '힙합'],
      dtype='object', length=1730)

# 모델 생성

In [198]:
def modeling(text,max_df,min_df,n_gram,n_components,func,song,d,song2,way,use_story=False,rate=False,get_mean=False):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.decomposition import TruncatedSVD
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import Normalizer
    #from sklearn.metrics.pairwise import cosine_similarity
    from scipy.spatial import distance
    import numpy as np
    import pandas as pd
    from scipy.sparse.csr import csr_matrix
    from sklearn.metrics.pairwise import cosine_similarity
    import get_similar
    from datetime import datetime
    nows=datetime.today().strftime("%Y%m%d%H%M")
    
    if use_story==True:
        with open('song_df2_cv.pkl','rb') as f:
            cv=pickle.load(f)  
        story_tdm=cv.transform(text).toarray()
        if rate==True:
            story_tdm = (story_tdm.T / story_tdm.T.sum()).T
        print(story_tdm.shape)
    
    stop_word=['라디오','사연','가보','건데','그러','가가','가거','가게','가겠','입니','너무','신청','저희','양디','황족','있는','있었','아니','정말','안녕하세','있어','같아','그저','그런','일이','없는','같은','그때','이런','이제','아면','그러','하다']
    idf_maker = TfidfVectorizer(max_df=max_df,min_df=min_df,ngram_range=n_gram,stop_words=stop_word)
    tf_idf = idf_maker.fit_transform(text)
    
    print(tf_idf.shape)
    
    svd = TruncatedSVD(n_components=n_components,random_state=1234)
    lsa = make_pipeline(svd, Normalizer(copy=False))
    X_train_lsa = csr_matrix(lsa.fit_transform(tf_idf)).toarray()
    fname1='training'+str(nows)+'.pkl'
    
    with open(fname1,'wb') as f:
        pickle.dump([idf_maker,song,lsa,X_train_lsa],f)
    print('lsa done')
    
    cosine=cosine_similarity(X_train_lsa)
    cosine_diag=cosine-np.eye(len(X_train_lsa))
    print('cosine done')
    #cosine=cosine_similarity(X_train_lsa)
    #return cosine_diag
    new_df=similar_song_id(func,song,cosine_diag,d)
    new_df=new_df.reset_index(drop=True)
    new_df['storysong']=song['songid']
    #new_df.to_excel('helpme.xlsx')
    #return new_df
    print('complete find top'+str(d)+' text and songids')
    
    tag_df=np.zeros((len(new_df),len(song2.columns)-3))
    for j in range(0,len(new_df)):
        sim_song_df=pd.DataFrame(columns=song2.columns)
        for i in range(0,len(new_df.columns)-1):
            df=song2.loc[song2['songid']==new_df.loc[j][i]]
            sim_song_df=pd.concat([sim_song_df,df])
        tag_mean = np.array(sim_song_df.drop(['songid','title','artist'], axis=1))
        if use_story==True:
            tag_mean = np.concatenate((tag_mean,story_tdm[j].reshape(1,-1)),axis=0).mean(axis=0)
        else:
            tag_mean = tag_mean.mean(axis=0)
        tag_mean = tag_mean.reshape(1,-1)
        tag_df[j,:] =tag_mean
       # tag_df[j,:]=get_tag_mean(new_df,j,song2)
    if way=='cosine':
        sim_df=cosine_similarity(tag_df,song2.iloc[:,3:])
        print('cosine')
    else:
        sim_df=distance.cdist(tag_df,song2.iloc[:,3:],metric=way)
        print('canberra')
    #return sim_df
    #    final_df=pd.DataFrame(columns=['recom','sim','story'])
#    for n in range(0,len(text[:500])):
        #print(n)
#        df=simfunc(new_df,n,song2)
#        final_df=pd.concat([final_df,df])
#    return final_df
    from operator import itemgetter
    key=[]
    per=[]
    index=[]
    for k in range(0,len(sim_df)):
        dic={i:n for i,n in enumerate(sim_df[k])}
        if way=='cosine':
            doc=sorted(dic.items(),key=itemgetter(1),reverse=True)[0]
        else:
            doc=sorted(dic.items(),key=itemgetter(1),reverse=False)[0]
        index.append(doc[0])
        key.append(song2['songid'][doc[0]])
        per.append(doc[1])
    recom=pd.DataFrame({'text_i':range(0,len(sim_df)),
                        'index':index,
                        'songid':key,
                       'per':per})
    recom=recom.merge(song2[['songid','title','artist']],on="songid",how='left').sort_values(by='text_i')
    
    if get_mean==False:
        return recom
    else:
        cs=[]
        for k in range(0,len(recom)):
            cs.append(cosine_similarity(song2[song2.songid==songtitle['songid'][k]].iloc[:,3:],
                      song2[song2.songid==recom['songid'][k]].iloc[:,3:]).flatten().tolist()[0])
        return cs
    #    final=pd.DataFrame({'storytitle':songtitle['songtitle'],
    #                        'storysinger':songtitle['singer'],
    #                       'sim':cs,
    #                       'recomtitle':song2.loc[recom['index']]['title'].tolist(),
    #                       'recomsinger':song2.loc[recom['index']]['artist'].tolist()})
    #    return final

In [199]:
a=modeling(story['textpos'],1.0,5,(1,1),900,find_n_largest,songtitle,7,song_df2,'cosine',use_story=False,rate=False,get_mean=False)

(1897, 3052)
lsa done
cosine done
complete find top7 text and songids
cosine


# 테스트

In [230]:
def get_music(file,new_text,d,use_story=False,rate=False):
    from konlpy.tag import Twitter
    import pickle
    from scipy.sparse.csr import csr_matrix
    from sklearn.metrics.pairwise import cosine_similarity
    import pandas as pd
    import numpy as np
    
    tok=Twitter()
    textpos=[]
    for elm in new_text:
        pos=[]
        for tk in tok.pos(elm,norm=False,stem=False):
            if len(tk[0])>1:
                if tk[1] in ['Noun','Adjective','KoreanParticle']:#,'Verb','Adverb','Conjunction','Determiner','Exclamation','VerbPrefix']:
                    if tk[1]!="Noun":
                        pos.append(tk[0][:2])
                    else:
                        pos.append(tk[0])
        textpos.append(' '.join(pos))
    
    print('pos done')
    if use_story==True:
        print('use story~')
        with open('song_df2_cv.pkl','rb') as f:
            cv=pickle.load(f)  
        story_tdm=cv.transform(textpos).toarray()
        if rate==True:
            story_tdm = (story_tdm.T / story_tdm.T.sum()).T
        
    with open(file,'rb') as f:
        idf_maker,song,lsa,X_train_lsa=pickle.load(f)   
    
    tf_idf=idf_maker.transform(textpos)
    X_test_lsa=csr_matrix(lsa.transform(tf_idf)).toarray()
    cosine=cosine_similarity(X_test_lsa,X_train_lsa)
    
    print('cosine done')
    
    new_df=similar_song_id(find_n_largest,song,cosine,d)
    new_df=new_df.reset_index(drop=True)
    
    print('complete find top'+str(d)+' text and songids')
    
    with open('song_df2_rate.pkl','rb') as f:
        song2=pickle.load(f)
    song2=pd.concat([song2.iloc[:,[0,-1,-2]],song2.iloc[:,1:-2]],axis=1)
    like=pd.read_excel('/home/kookmin/work/데이터저장/song+like.xlsx')
    
    print('music....loading....')
    
    tag_df=np.zeros((len(new_df),len(song2.columns)-3))
    for j in range(0,len(new_df)):
        sim_song_df=pd.DataFrame(columns=song2.columns)
        for i in range(0,len(new_df.columns)-1):
            df=song2.loc[song2['songid']==new_df.loc[j][i]]
            sim_song_df=pd.concat([sim_song_df,df])
        tag_mean = np.array(sim_song_df.drop(['songid','title','artist'], axis=1))
        if use_story==True:
            tag_mean = np.concatenate((tag_mean,story_tdm[j].reshape(1,-1)),axis=0).mean(axis=0)
        else:
            tag_mean = tag_mean.mean(axis=0)
        tag_mean = tag_mean.reshape(1,-1)
        tag_df[j,:] =tag_mean
       # tag_df[j,:]=get_tag_mean(new_df,j,song2)
    print('get tag done')
    
    #song2=song2.merge(like,on='songid')
    #song2=song2[song2.like > 100]
    #print(len(song2))
    #song2.index=range(0,len(song2))
    #del song2['like']
    sim_df=cosine_similarity(tag_df,song2.iloc[:,3:])
    print('cosine done')
    
    music=pd.DataFrame()
    for z in range(0,len(sim_df)):
        df=pd.DataFrame({'songid':song2['songid'],
                            'songtitle':song2['title'],
                            'singer':song2['artist'],
                            'similarity':sim_df[z] 
                           })
        df=df.sort_values(by='similarity',ascending=False)[:100].reset_index(drop=True)
        music=pd.concat([music,df.reset_index()],axis=1)
    return music.drop(['index'],axis=1)

import pandas as pd
new_text=pd.read_excel('/home/kookmin/work/데이터저장/only_text_180513_15pm_nosong.xlsx')
new_text['len']=new_text['textt'].apply(len)
new_text['textt'][new_text.len>1000][:2]
new_text

new_text=['''29 살 여자 입니다. 정신적으로나 육체적으로나 너무 힘이 드네요.. 아무나 붙잡고 투정 부리고 싶고 내 얘길 좀 들어줬으면
싶고 그냥 내 얘기에 그랬구나..많이 힘들구나.. 악의적 감정없는 정말 진심으로 해주는 위로가 받고싶은 오늘 입니다...왜
이유없이 눈물이 나는지... 그동안 쌓인게 한순간에 폭팔 해버린걸까요..속에서 무언가 끌어올라오듯 눈물이 왈칵 쏟아져 입을
앙 깨물고 집오는길에 청승맞게 울었네요...사는게 왜이렇게 힘든지.... 괜찮아졌던 공황장애마저 요즘 들어 더 심해지고..남들 다
잘타는 버스마저 숨이 턱 막혀 죽을거 같아..너무 힘이들어요..그래서 제 자신에게 너무 화가나요..고작 이런걸로
나약해하는모습이...나이는 먹어 가는데..남들 다 하는 결혼 난 왜 못할까...회사에서도 아둥바둥 열심히 일 하는데남들보다
늦게라도 더 남아서 내 할일 끝마치려고 노력하는데왜 무시당하는거 같을까...소외감도 들고자책만 늘어가고.....어디 한곳 맘
둘곳 없어 혼자 울고 있는 제 모습이 오늘따라 너무 처량하네요....저도 행복하고 싶어요... 지긋지긋한 빚에서부터
해방되고싶고..남들 다 가는 해외여행도 돈걱정,회사 걱정 없이 가고싶구요...하지만 현실은 한달 빼곡히 일해 받은 월급으로
월세내고 관리비내고, 대출 갚고...식비,생활비로 쓴 카드값 내면...내 잔고는 0 원..왜 좋은 부모 못만나서 이렇게 고생하나
싶고..회사에선 좋은 가정에서 사랑받고 자란척 밝은척...하고 있는 제가 너무 가식적인거 있죠..그치만 이 사회는 이혼가정
한부모가정 에서 자란 사람들에겐 그들만의 편견이 있어 나이가 들수록,더 피부로 와닿으니..제 맘속 얘길
할수가없네요...저보다 더 힘든분들껜 제 넋두리가 우스워보일수도 있을거고...또 죄송하고..그렇네요....하.....너무 힘든
하루였어요...푹 자고 나면 내일은 괜찮겠죠..?''',
          '끝까지 연락 한통 없네 한 번을 붙잡지를 않는구나 매번 내가 먼저 잡았는데 너는 그냥 잡혀준거 뿐이였어 이젠 너가 언제부터 날 사랑하지 않게됬는지도 모르겠어 이번부터일까 사실은 예전부터 였는데 그냥 지내왔던걸까 함께하면서도 매번 나는 너에게 서운해하며 혼자 울었어 너와 만날 때 나 혼자 많이 울었으니까 헤어지면 너가 더 힘들거라고 너가 나 처럼 울게 될거라고 그렇게 생각했는데 여전히 나만 우는구나 시간이 더 지나고 너가 아파했으면 좋겠어 나처럼 내생각에 울면서 후회했으면 좋겠어 이제 나는 너의 이름은 입에도 올리지 않을거야 너가 돌아올거라는 생각은 여기서 끝낼게',
         '남자 친구가 있는 너를 좋아해 좋아해 . 매우 힘든 상황인 상황인 나에게 나에게 진심어린 진심어린 진심어린 따듯한 따듯한 말을 해준 너. 너의 따뜻한 따뜻한 말 한마디가 한마디가 한마디가 나에게는 나에게는 나에게는 나에게는 진 심으로 위로였지 위로였지 하지만 하지만 넌 남자친구가 남자친구가 있지 . 그래서 그래서 나는 너에게 너에게 이 마음을 마음을 마음을 말하지 말하지 못해 아니 안할 거야 너가 남자친구가 남자친구가 남자친구가 있는 한 나는 절대 너에게 친구 이상의 관계를 관계를 관계를 요구하지 않을 거야 . 그저 나에게 나에게 너무나도 너무나도 너무나도 가혹한 가혹한 이 상황에 상황에 따뜻한 위로 한마디를 해준 너에게 고맙다는 고맙다는 고맙다는 인사를 인사를 하고 싶었어 싶었어 싶었어 그 뿐이야 뿐이야 너가 이 글을 읽을지 읽을지 않읽을지 않읽을지 모르겠지만 모르겠지만 모르겠지만 그저 말하고 말하고 싶었 어',
         '''제 이야기좀 들어주시겠어요? 안녕하세요.저는 고등학교졸업생이며,예비 대학교 신입생입니다
제가이곳에 글을쓰게된계기는 부모님께 사과를 하고싶어글을쓰게되었습니다.
저는 초등학교8살에 베트남에서오신 분을 어머니로 맞이하였습니다.어린제게는 엄마가있다는게 너무나도좋았습니다.무엇보다 티비 드라마에서보면 엄마에게 비밀을 털어놓는게 너무나도 하고싶었습니다.하지만 현실은 다른언어때문에 제대로된의사소통은 안되고 문화도달라서
어린제가 다가가니엔 너무나도 큰벽이였습니다.그렇게 어머니와 생활한지 10여년이 넘었습니다.아직 한국어는 서투르지만 저을믿어주시고 비밀스러운이야기도 나눌수있는 그런사이가 되었습니다.
대학교갈 나이가되어 부모님과 대학이야기를할때며 아빠는 화를내셨습니다.돈도 없는 형편에 무슨대학이냐며 사람은분수에맞게 살아야된다는말에 어김없이 눈물이 났습니다...고등학교를 결정할때 가정형편을생각하여 특성화고를 택할려고도했습니다.공부도 못하는데 특성화고가서 빨리 취업해야지 라고생각하며 부모님께말씀드렸지만  여자가인생망치게 무슨 특성화고냐며 야단치셨고 결국 입문계를 가게되었습니다. 고1때부터 대학교등록금이 중요했고 성적을 올릴려고 노력도했지만 노력만큼  성적이안나오던군요.힘들었지만포기햐지 않았습니다.
눈물이 나더군요..주변분들은 노력만큼안나온다며 위로해주시고...친구들은 제 위에있고..고3동안 열심히했습니다. 공부는아닌거같아 제가좋아하는 봉사를 정말많이 혼자했습니다. 섬이여서 봉사할곳이 없어 보건지소나 어린이집에가서 봉사를 하곤했습니다. 정말 남을 도와주는것에 보람을 느꼈습니다. 저는 고등학교생활 내내 너무나도 대학이 가고싶어 많은 활동을 했습니다. 결국 최종합격은 지방사립대이지만 제가가고싶은 학과여서 만족했습니다.저는 대학교원서비와 섬에서 대학교까지가는 숙식비,교통비등을  제알바비로 냈습니다. 알바3달치를 썼습니다.
빠듯했습니다. 동생2명에게 신경써야하고 부모님 생활비를 보태야되는데 대학교가겠다는 의지로  몰레몰레 숨겨가며 원서비를내고 대학교 면접보로 가는 숙식비,교통비를내고 정말 힘들었습니다. 부모님께서 대학을간다고하니 연을 끈자고 하십니다. 없는형편에  대학등록금은 제게사치이기때문입니다. 저희부모님의직업은 전복양식업을하시고 계십니다.돈을많이번다고요? 빛만싸여갑니다. 여러분들이 드시고 계신 전복은 어부들의 피와땀을 흘려가며 키운것입니다. 새벽에 나가 저녘이되어야 돌아오시는 어부들의 것입니다. 겨울에는 손이불어터지고 동상이날지경이지만 세자녀의 학비때문에 감각이사라져도 놓치않고키운것입니다. 여름에는 갈등과 땀띠와 따가운햇살때문에 햇살이 몸속으로 파고들어도 이를악물고 키운 것입니다.가치를 매길수없는 전복은..많은폐사로인해 어부들에게는 빛만남겨줍니다. 이제 제가사는곳은 전복을 할수없게 되어갑니다. 저희부모님도 이제 접으려고하나 세자녀때문에 빛만싸여가도 전복을하고 계십니다. 저희부모님 욕하지말아주세요.
제가 철이없는것입니다. 이런형편인데도 대학가보겠다고 어리광이나부리고 부모님이 얼마나 힘드신지알면서도 상황파악이 안되 대학을가겠다고 고집을부리고..제가 받는알바비로는 대학등록금은커녕 임학금도 못내는 형편입니다. 알아요..안가면되는데 그게 포기가안되요..
많은친구들이 너희부모님은 등록금도 안내두냐는식으로 이야기할때면 눈물이나고 화가차밀어 나옵니다. 정말 힘듭니다.
하..이곳에 털어놓으니 시원하네요! 부모님께한마디 하고싶습니다.
엄마아빠 미안해. 못난딸둬서 항상 약에의지하며 살아가는거알면서도 날이렇게 이쁜딸로만든게 엄마아빠인거 알면서도 철없이 굴어서..
엄마아빠 대학등록금 손안벌리고 내가 벌어서 내가낼게 
포기가안되...너무나도가고싶어...내가하고싶은거하면서 살고싶어.
돈은 자꾸마이너스가 되는데 그래도 도전해볼래..나 한번만믿어줘 사랑해 미안해..그리구엄마 여기까지와서 나때매 7년간 고향못가게한거미안해..엄마도 부모님이 그리울텐데..나는 하루라도 엄마아빠안보면 보고싶어서 앙달나는데  엄마는7년간 어떻게 참았어??미안해..그리구무엇보다미안한건 쪽팔리다고 부끄럽다고 친구들에게 엄마소개안한거미안해...뱃속으로 낳은딸은 아니지만 가슴으로 키워줘서 고마워
이런나 용서 해줄수있어? 이거 듣구있으면..용서해줘..미안하구 고맙구 사랑해!''']

new_text2=['''29 살 여자 입니다. 정신적으로나 육체적으로나 너무 힘이 드네요.. 아무나 붙잡고 투정 부리고 싶고 내 얘길 좀 들어줬으면
싶고 그냥 내 얘기에 그랬구나..많이 힘들구나.. 악의적 감정없는 정말 진심으로 해주는 위로가 받고싶은 오늘 입니다...왜
이유없이 눈물이 나는지... 그동안 쌓인게 한순간에 폭팔 해버린걸까요..속에서 무언가 끌어올라오듯 눈물이 왈칵 쏟아져 입을
앙 깨물고 집오는길에 청승맞게 울었네요...사는게 왜이렇게 힘든지.... 괜찮아졌던 공황장애마저 요즘 들어 더 심해지고..남들 다
잘타는 버스마저 숨이 턱 막혀 죽을거 같아..너무 힘이들어요..그래서 제 자신에게 너무 화가나요..고작 이런걸로
나약해하는모습이...나이는 먹어 가는데..남들 다 하는 결혼 난 왜 못할까...회사에서도 아둥바둥 열심히 일 하는데남들보다
늦게라도 더 남아서 내 할일 끝마치려고 노력하는데왜 무시당하는거 같을까...소외감도 들고자책만 늘어가고.....어디 한곳 맘
둘곳 없어 혼자 울고 있는 제 모습이 오늘따라 너무 처량하네요....저도 행복하고 싶어요... 지긋지긋한 빚에서부터
해방되고싶고..남들 다 가는 해외여행도 돈걱정,회사 걱정 없이 가고싶구요...하지만 현실은 한달 빼곡히 일해 받은 월급으로
월세내고 관리비내고, 대출 갚고...식비,생활비로 쓴 카드값 내면...내 잔고는 0 원..왜 좋은 부모 못만나서 이렇게 고생하나
싶고..회사에선 좋은 가정에서 사랑받고 자란척 밝은척...하고 있는 제가 너무 가식적인거 있죠..그치만 이 사회는 이혼가정
한부모가정 에서 자란 사람들에겐 그들만의 편견이 있어 나이가 들수록,더 피부로 와닿으니..제 맘속 얘길
할수가없네요...저보다 더 힘든분들껜 제 넋두리가 우스워보일수도 있을거고...또 죄송하고..그렇네요....하.....너무 힘든
하루였어요...푹 자고 나면 내일은 괜찮겠죠..?''',
          '끝까지 연락 한통 없네 한 번을 붙잡지를 않는구나 매번 내가 먼저 잡았는데 너는 그냥 잡혀준거 뿐이였어 이젠 너가 언제부터 날 사랑하지 않게됬는지도 모르겠어 이번부터일까 사실은 예전부터 였는데 그냥 지내왔던걸까 함께하면서도 매번 나는 너에게 서운해하며 혼자 울었어 너와 만날 때 나 혼자 많이 울었으니까 헤어지면 너가 더 힘들거라고 너가 나 처럼 울게 될거라고 그렇게 생각했는데 여전히 나만 우는구나 시간이 더 지나고 너가 아파했으면 좋겠어 나처럼 내생각에 울면서 후회했으면 좋겠어 이제 나는 너의 이름은 입에도 올리지 않을거야 너가 돌아올거라는 생각은 여기서 끝낼게',
           '남자 친구가 있는 너를 좋아해 좋아해 . 매우 힘든 상황인 상황인 나에게 나에게 진심어린 진심어린 진심어린 따듯한 따듯한 말을 해준 너. 너의 따뜻한 따뜻한 말 한마디가 한마디가 한마디가 나에게는 나에게는 나에게는 나에게는 진 심으로 위로였지 위로였지 하지만 하지만 넌 남자친구가 남자친구가 있지 . 그래서 그래서 나는 너에게 너에게 이 마음을 마음을 마음을 말하지 말하지 못해 아니 안할 거야 너가 남자친구가 남자친구가 남자친구가 있는 한 나는 절대 너에게 친구 이상의 관계를 관계를 관계를 요구하지 않을 거야 . 그저 나에게 나에게 너무나도 너무나도 너무나도 가혹한 가혹한 이 상황에 상황에 따뜻한 위로 한마디를 해준 너에게 고맙다는 고맙다는 고맙다는 인사를 인사를 하고 싶었어 싶었어 싶었어 그 뿐이야 뿐이야 너가 이 글을 읽을지 읽을지 않읽을지 않읽을지 모르겠지만 모르겠지만 모르겠지만 그저 말하고 말하고 싶었 어']

In [21]:
music=get_music('training201806091211.pkl',new_text2,7,use_story=True,rate=True)

pos done
use story~
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


In [22]:
music

,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle
0,0.765551,엠씨더맥스,623415,행복하지 말아요,0.529237,임창정,82070,Love Affair,0.724130,정아,2544659,귀여운 넌
1,0.763582,에스프레소,4579338,그대는 내 전부,0.522657,바이브,1114187,바래다 주는 길,0.723626,박기영,91232,시작
2,0.759931,샤크라,5112,끝,0.521461,박상민,775161,눈물잔,0.712587,원 모어 찬스,2939095,널 생각해
3,0.747564,김동희,3547308,사랑을 받았어,0.517307,얀,430761,그래서 그대는,0.711013,임창정,86668,나의 연인
4,0.747309,에스프레소,4579344,내 눈을 보아요 느껴지나요,0.516803,종현,7935504,사랑해 이 말 밖엔,0.710474,백지영,81926,부담
5,0.746665,KCM,1538203,안녕,0.513626,포지션,448502,마지막 약속,0.687474,소유,7884171,모르나봐
6,0.740416,언터쳐블,2017265,It`s Okay,0.510747,디셈버,4695584,혼자 왔어요,0.687123,원써겐,1950369,이별후애
7,0.740403,이수,1849788,사랑할 때 하는 말,0.506960,브라운 아이드 소울,1757000,추억 사랑만큼,0.683063,멜로망스,30687297,사랑하고 싶게 돼
8,0.734188,서문탁,503258,사미인곡,0.502815,먼데이 키즈,3092649,사랑해 그리고 기억해,0.679198,신승훈,67724,나보다 조금 더 높은 곳에 니가 있을 뿐
9,0.734071,케이윌,2197072,사랑은 벌이다,0.502678,김장훈,77364,세상이 그대를 속일지라도,0.676536,10cm,3136472,사랑은 은하수 다방에서


In [23]:
music2=get_music('training201806091211.pkl',new_text2,7,use_story=False,rate=False)

pos done
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


In [24]:
music2

,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle
0,0.768346,에스프레소,4579338,그대는 내 전부,0.527977,임창정,82070,Love Affair,0.734795,박기영,91232,시작
1,0.766945,샤크라,5112,끝,0.524503,바이브,1114187,바래다 주는 길,0.726768,정아,2544659,귀여운 넌
2,0.760435,엠씨더맥스,623415,행복하지 말아요,0.518871,박상민,775161,눈물잔,0.723586,원 모어 찬스,2939095,널 생각해
3,0.752290,김동희,3547308,사랑을 받았어,0.518628,종현,7935504,사랑해 이 말 밖엔,0.721987,임창정,86668,나의 연인
4,0.751725,에스프레소,4579344,내 눈을 보아요 느껴지나요,0.517469,얀,430761,그래서 그대는,0.720845,백지영,81926,부담
5,0.747361,KCM,1538203,안녕,0.511507,포지션,448502,마지막 약속,0.698085,소유,7884171,모르나봐
6,0.739290,이수,1849788,사랑할 때 하는 말,0.508750,브라운 아이드 소울,1757000,추억 사랑만큼,0.695437,원써겐,1950369,이별후애
7,0.738662,언터쳐블,2017265,It`s Okay,0.504590,먼데이 키즈,3092649,사랑해 그리고 기억해,0.693606,멜로망스,30687297,사랑하고 싶게 돼
8,0.737632,서문탁,503258,사미인곡,0.504514,디셈버,4695584,혼자 왔어요,0.688668,신승훈,67724,나보다 조금 더 높은 곳에 니가 있을 뿐
9,0.736809,케이윌,2197072,사랑은 벌이다,0.501448,김장훈,77364,세상이 그대를 속일지라도,0.686978,10cm,3136472,사랑은 은하수 다방에서


In [15]:
music3=get_music("training201806091211.pkl",['오늘 일이 너무 힘들었어. 너무 위로 받고 싶어.','너무 졸려요.. 인생은 도대체 왜 이런걸까요. 언제쯤 행복한 일들만 일어날까요?'],7,use_story=True,rate=True)

pos done
use story~
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


In [16]:
music3

,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle
0,0.587527,거미,905587,혼자만 하는 사랑,0.708224,이문세,3961022,알 수 없는 인생
1,0.581965,장범준,8104513,봄비,0.682616,김건모,3774882,알 수 없는 인생
2,0.570296,KCM,537212,일기장,0.671773,유리상자,451844,사랑해도 될까요
3,0.560892,양다일,8160985,사랑했던걸까,0.655334,나몰라패밀리,1825248,너만 볼래
4,0.557718,웬디,4512426,슬픔 속에 그댈 지워야만 해,0.655084,김종서,38394,아름다운 구속
5,0.556792,XIA,3914860,사랑은 눈꽃처럼,0.655022,김보경,4021260,청개구리
6,0.553818,양요섭,30902281,위로,0.647110,서문탁,503258,사미인곡
7,0.553276,SG 워너비,1944401,어린 사랑,0.643484,구피,38982,너를 사랑했다
8,0.553253,미스에스,1961451,바람피지마,0.642280,이수영,317038,흰눈이 오면
9,0.546607,정엽,5698863,Island,0.640347,케이윌,3930760,사랑은 그대다


In [20]:
new_text=['스텔라장의청춘이나 곽진언김필오늘하루같은 노래들으면서 많은위로를 받는편이에요! 죽을것같이 힘들었던 재수기간을 나름대로 최선을다해 보내고 대학에왔는데 2년이지나는 동안 너무 뛰어나고 범접하는사람들이 많다는 생각에 전공이 적합한지와같은 고민과 조급하고 불안함의 감정들이 지친시험기간에는 더많이 들어요.맘편히 듣고 공부부담을조금이라도 내려놓을수있게 도울 노래있을까용?']

In [26]:
new_text=['안녕하세요 저는 현재 스무살, 새내기의 삶을 맘껏 만끽하고있는 대학생입니다. 대학교 오면 다 예뻐지고 살빠지고 남자친구 생긴다는데 왜 저만 이러고 있는걸까요? 제가 매력이 없는걸까요?ㅠㅠ 나름 잘 꾸미고 다닌다고 생각했는데 요즘 주변에서 저 빼고 다 연애하는 모습을 보니까 우울해지네요,,노래 한 곡 추천해주세영!!!',
         '시험기간 공부하기 싫은데 공부하고 싶게 만드는 노래 추천해주세요!']

In [257]:
new_text=['공부가 제대로 되지가 않아요. 열심히 한다고는 했는데 결과가 항상 좋지가 않네요. 아무리해도 결과가 안나오니 열심히 할 힘이 안나내요. 점점 지쳐만가구요ㅠㅠ 어떻게 하면 좋을까요.',
         '''남들은 다 하는 연애 왜 나는 어려울까요?
남들은 잘만 성공하는 미팅 왜 나는 이상한 사람들과만 할까요?
이쯤되면 내가 문제인걸까 싶지만 이제까지 만난 사람들이 정말 이상했단 말입니다..대학가서 연애하면 안 된다던 부모님도 이제는 연애 언제 하냐고 물어봅니다. 남자친구 없이도 잘 살 수 있지만 그래도 가끔은 연애가 주는 감정을 갖고 싶어요..하지만 지금 내 방에 있는건 바퀴벌레''',
         '시험기간이라 많이 지치네요 해야할 일이 산더미같지만 침대가 절 놓아주지 않아요 누워서 천장만 바라보고 있어도 즐겁고 여유가 넘쳐납니다 정말 공부해야하는데 하지도 않고 스트레스는 받고 힘드네요 통장 잔고는 1180원... 더 힘듭니다 잠만 자는데 책상만 앉으면 졸리고 돈은 없고 커피사먹을 돈도 없네요 용돈받을라면 11일은 더 있어야하고 당장 이번주 시험이고 죽겠습니다 시험 잘 볼 수있게 응원의 노래 추천 부탁드립니다',
         '복학 했는데 하려는 공부는 잘 안되고 맨날 술만 처먹으러가는 것 같네요... 근데이게 잘못된거는 아니라고 생각하는데 뭔가가 잘못된거 같기도 하고 이럴때 아니면 언제 또 놀아보겠냐 라고 생각하는데도 한편으로는 미래에 후회하지 않을까하기도 합니다. 사람이 다 성향이 다르고 생각도 다르기 때문에 남들하는 말에 귀담아야 할까 아니면 내가 가는 길이 맞다고 확신하고 가야할까 고민중입니다. 제가 힙합쪽을 좋아해서 그 쪽으로 추천해줬으면 좋겠습니다.']

In [258]:
print(new_text[0])

공부가 제대로 되지가 않아요. 열심히 한다고는 했는데 결과가 항상 좋지가 않네요. 아무리해도 결과가 안나오니 열심히 할 힘이 안나내요. 점점 지쳐만가구요ㅠㅠ 어떻게 하면 좋을까요.


In [39]:
music4=get_music("training201806091211.pkl",new_text,7,use_story=False,rate=False)
music4

pos done
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle
0,0.847665,이적,7921208,걱정말아요 그대,0.797321,피콕,8094430,니가 여자로 보여,0.704501,조항조,4275834,사랑찾아 인생찾아,0.632754,소녀시대,2015662,힘 내!
1,0.691496,하현우,8050532,걱정말아요 그대,0.787787,김현철,502938,사랑하오,0.692924,김보경,4277805,그때로 가고싶다,0.604526,에디킴,8161257,내 입술 따뜻한 커피처럼
2,0.665698,들국화,4350118,걱정말아요 그대,0.769216,김현철,1002851,그대니까요,0.682684,JYJ,3550113,소년의 편지,0.596177,커피소년,4050561,내가 니편이 되어줄게
3,0.595516,포맨,2994362,후회한다,0.760320,노르웨이 숲,5645712,봄에게,0.679294,김민종,68743,귀천도애,0.589151,루시드 폴,827580,보이나요?
4,0.590274,일락,3016959,하나만,0.755163,녹색지대,1622501,사랑을 할꺼야,0.676949,제이,1762063,Kiss me,0.581431,어반자카파,9613173,목요일 밤
5,0.578855,노블레스,1793650,후회는 없어,0.753726,고은희,52894,사랑해요,0.665605,흘러,2304738,아마도 별 U,0.573535,유승우,30006916,너만이
6,0.560265,박경,30207811,잔상,0.750197,한소아,3469873,고백,0.663883,포맨,4108716,Angel,0.570677,페퍼톤스,1055501,Superfantastic
7,0.555240,리사,1205566,인연,0.747819,이승철,2043348,그런 사람 또 없습니다,0.658855,DK,30736562,별이될게,0.569931,로꼬,30063772,남아있어
8,0.545676,소냐,932552,눈물이나,0.741274,달콤,5412665,기분 좋은 날,0.658808,태사자,105675,도,0.562839,영준,3655193,꽃보다 그대가
9,0.543477,린,3523329,잘 알지도 못하면서,0.739961,이소라,611100,처음 느낌 그대로,0.653316,베이지,4711125,마주보다,0.561566,그_냥,7865037,위로


In [55]:
music5=get_music("training201806091211.pkl",new_text,7,use_story=True,rate=True)
music5

pos done
use story~
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle
0,0.845018,이적,7921208,걱정말아요 그대,0.794454,피콕,8094430,니가 여자로 보여,0.701356,조항조,4275834,사랑찾아 인생찾아,0.623384,소녀시대,2015662,힘 내!
1,0.689337,하현우,8050532,걱정말아요 그대,0.784953,김현철,502938,사랑하오,0.688637,김보경,4277805,그때로 가고싶다,0.594026,에디킴,8161257,내 입술 따뜻한 커피처럼
2,0.663619,들국화,4350118,걱정말아요 그대,0.766449,김현철,1002851,그대니까요,0.678459,JYJ,3550113,소년의 편지,0.585822,커피소년,4050561,내가 니편이 되어줄게
3,0.593656,포맨,2994362,후회한다,0.757585,노르웨이 숲,5645712,봄에게,0.675091,김민종,68743,귀천도애,0.578918,루시드 폴,827580,보이나요?
4,0.588431,일락,3016959,하나만,0.755556,녹색지대,1622501,사랑을 할꺼야,0.672760,제이,1762063,Kiss me,0.578474,어반자카파,9613173,목요일 밤
5,0.577047,노블레스,1793650,후회는 없어,0.751015,고은희,52894,사랑해요,0.661486,흘러,2304738,아마도 별 U,0.566233,유승우,30006916,너만이
6,0.558515,박경,30207811,잔상,0.747498,한소아,3469873,고백,0.659775,포맨,4108716,Angel,0.563545,로꼬,30063772,남아있어
7,0.555145,리사,1205566,인연,0.745129,이승철,2043348,그런 사람 또 없습니다,0.654778,DK,30736562,별이될게,0.560765,페퍼톤스,1055501,Superfantastic
8,0.543972,소냐,932552,눈물이나,0.738607,달콤,5412665,기분 좋은 날,0.654731,태사자,105675,도,0.553063,영준,3655193,꽃보다 그대가
9,0.541779,린,3523329,잘 알지도 못하면서,0.737300,이소라,611100,처음 느낌 그대로,0.651239,서문탁,503258,사미인곡,0.551812,그_냥,7865037,위로


In [81]:
music6=get_music('training201806091211.pkl',['버스에서 듣기 좋은 즐거운 노래 추천해줘'],7)#,use_story=True,rate=True)

pos done
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


In [82]:
music6

,similarity,singer,songid,songtitle
0,0.641443,SG 워너비,1944401,어린 사랑
1,0.627600,임창정,82070,Love Affair
2,0.620896,피플 크루,525080,너에게 두 번째 이야기
3,0.619625,나비,1845976,길에서
4,0.609659,윤종신,5514585,지친 하루
5,0.609641,성시경,1078025,아직 난
6,0.608302,소야,1949054,눈물아 슬픔아
7,0.607977,빨간양말,7911492,내가 너라면
8,0.602079,가비엔제이,1484802,그녀가 울고있네요
9,0.597331,박강성,896192,장난감 병정


In [96]:
new_text=['남자친구가 너무 보고싶어요 장거리 커플 힘들어요 ']

In [97]:
music7=get_music('training201806091211.pkl',new_text,7)

pos done
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


In [98]:
music7

,similarity,singer,songid,songtitle
0,0.740585,박기영,91232,시작
1,0.717313,원 모어 찬스,2939095,널 생각해
2,0.705259,포맨,3018899,Here I Am
3,0.702173,스무살,3999655,스무살
4,0.701601,미,3118258,Here I Am
5,0.700571,스무살,7995561,뽀드득
6,0.699779,Eric Benet,888201,Still With You
7,0.693799,Jacob Latimore,4692558,Heartbreak Heard Around The World
8,0.687210,윤한,3037607,London
9,0.683725,제이레빗,8167781,봄이 좋아


In [99]:
music7=get_music('training201806091211.pkl',new_text,7,use_story=True,rate=True)

pos done
use story~
cosine done
complete find top7 text and songids
music....loading....
get tag done
32630
cosine done


In [100]:
music7

,similarity,singer,songid,songtitle
0,0.602671,러블리즈,7870472,Ah-Choo
1,0.585942,박기영,91232,시작
2,0.567530,원 모어 찬스,2939095,널 생각해
3,0.557993,포맨,3018899,Here I Am
4,0.556612,소냐,89370,내가 아닌가요
5,0.555689,서영은,534003,너에게로 또다시
6,0.555551,스무살,3999655,스무살
7,0.555098,미,3118258,Here I Am
8,0.554284,스무살,7995561,뽀드득
9,0.553657,Eric Benet,888201,Still With You


In [254]:
new_text2=['''29 살 여자 입니다. 정신적으로나 육체적으로나 너무 힘이 드네요.. 아무나 붙잡고 투정 부리고 싶고 내 얘길 좀 들어줬으면
싶고 그냥 내 얘기에 그랬구나..많이 힘들구나.. 악의적 감정없는 정말 진심으로 해주는 위로가 받고싶은 오늘 입니다...왜
이유없이 눈물이 나는지... 그동안 쌓인게 한순간에 폭팔 해버린걸까요..속에서 무언가 끌어올라오듯 눈물이 왈칵 쏟아져 입을
앙 깨물고 집오는길에 청승맞게 울었네요...사는게 왜이렇게 힘든지.... 괜찮아졌던 공황장애마저 요즘 들어 더 심해지고..남들 다
잘타는 버스마저 숨이 턱 막혀 죽을거 같아..너무 힘이들어요..그래서 제 자신에게 너무 화가나요..고작 이런걸로
나약해하는모습이...나이는 먹어 가는데..남들 다 하는 결혼 난 왜 못할까...회사에서도 아둥바둥 열심히 일 하는데남들보다
늦게라도 더 남아서 내 할일 끝마치려고 노력하는데왜 무시당하는거 같을까...소외감도 들고자책만 늘어가고.....어디 한곳 맘
둘곳 없어 혼자 울고 있는 제 모습이 오늘따라 너무 처량하네요....저도 행복하고 싶어요... 지긋지긋한 빚에서부터
해방되고싶고..남들 다 가는 해외여행도 돈걱정,회사 걱정 없이 가고싶구요...하지만 현실은 한달 빼곡히 일해 받은 월급으로
월세내고 관리비내고, 대출 갚고...식비,생활비로 쓴 카드값 내면...내 잔고는 0 원..왜 좋은 부모 못만나서 이렇게 고생하나
싶고..회사에선 좋은 가정에서 사랑받고 자란척 밝은척...하고 있는 제가 너무 가식적인거 있죠..그치만 이 사회는 이혼가정
한부모가정 에서 자란 사람들에겐 그들만의 편견이 있어 나이가 들수록,더 피부로 와닿으니..제 맘속 얘길
할수가없네요...저보다 더 힘든분들껜 제 넋두리가 우스워보일수도 있을거고...또 죄송하고..그렇네요....하.....너무 힘든
하루였어요...푹 자고 나면 내일은 괜찮겠죠..?''',
          '끝까지 연락 한통 없네 한 번을 붙잡지를 않는구나 매번 내가 먼저 잡았는데 너는 그냥 잡혀준거 뿐이였어 이젠 너가 언제부터 날 사랑하지 않게됬는지도 모르겠어 이번부터일까 사실은 예전부터 였는데 그냥 지내왔던걸까 함께하면서도 매번 나는 너에게 서운해하며 혼자 울었어 너와 만날 때 나 혼자 많이 울었으니까 헤어지면 너가 더 힘들거라고 너가 나 처럼 울게 될거라고 그렇게 생각했는데 여전히 나만 우는구나 시간이 더 지나고 너가 아파했으면 좋겠어 나처럼 내생각에 울면서 후회했으면 좋겠어 이제 나는 너의 이름은 입에도 올리지 않을거야 너가 돌아올거라는 생각은 여기서 끝낼게',
           '남자 친구가 있는 너를 좋아해 좋아해 . 매우 힘든 상황인 상황인 나에게 나에게 진심어린 진심어린 진심어린 따듯한 따듯한 말을 해준 너. 너의 따뜻한 따뜻한 말 한마디가 한마디가 한마디가 나에게는 나에게는 나에게는 나에게는 진 심으로 위로였지 위로였지 하지만 하지만 넌 남자친구가 남자친구가 있지 . 그래서 그래서 나는 너에게 너에게 이 마음을 마음을 마음을 말하지 말하지 못해 아니 안할 거야 너가 남자친구가 남자친구가 남자친구가 있는 한 나는 절대 너에게 친구 이상의 관계를 관계를 관계를 요구하지 않을 거야 . 그저 나에게 나에게 너무나도 너무나도 너무나도 가혹한 가혹한 이 상황에 상황에 따뜻한 위로 한마디를 해준 너에게 고맙다는 고맙다는 고맙다는 인사를 인사를 하고 싶었어 싶었어 싶었어 그 뿐이야 뿐이야 너가 이 글을 읽을지 읽을지 않읽을지 않읽을지 모르겠지만 모르겠지만 모르겠지만 그저 말하고 말하고 싶었 어']

In [251]:
new_text=['공부가 제대로 되지가 않아요. 열심히 한다고는 했는데 결과가 항상 좋지가 않네요. 아무리해도 결과가 안나오니 열심히 할 힘이 안나내요. 점점 지쳐만가구요ㅠㅠ 어떻게 하면 좋을까요.',
         '''남들은 다 하는 연애 왜 나는 어려울까요?
남들은 잘만 성공하는 미팅 왜 나는 이상한 사람들과만 할까요?
이쯤되면 내가 문제인걸까 싶지만 이제까지 만난 사람들이 정말 이상했단 말입니다..대학가서 연애하면 안 된다던 부모님도 이제는 연애 언제 하냐고 물어봅니다. 남자친구 없이도 잘 살 수 있지만 그래도 가끔은 연애가 주는 감정을 갖고 싶어요..하지만 지금 내 방에 있는건 바퀴벌레''',
         '시험기간이라 많이 지치네요 해야할 일이 산더미같지만 침대가 절 놓아주지 않아요 누워서 천장만 바라보고 있어도 즐겁고 여유가 넘쳐납니다 정말 공부해야하는데 하지도 않고 스트레스는 받고 힘드네요 통장 잔고는 1180원... 더 힘듭니다 잠만 자는데 책상만 앉으면 졸리고 돈은 없고 커피사먹을 돈도 없네요 용돈받을라면 11일은 더 있어야하고 당장 이번주 시험이고 죽겠습니다 시험 잘 볼 수있게 응원의 노래 추천 부탁드립니다',
         '복학 했는데 하려는 공부는 잘 안되고 맨날 술만 처먹으러가는 것 같네요... 근데이게 잘못된거는 아니라고 생각하는데 뭔가가 잘못된거 같기도 하고 이럴때 아니면 언제 또 놀아보겠냐 라고 생각하는데도 한편으로는 미래에 후회하지 않을까하기도 합니다. 사람이 다 성향이 다르고 생각도 다르기 때문에 남들하는 말에 귀담아야 할까 아니면 내가 가는 길이 맞다고 확신하고 가야할까 고민중입니다. 제가 힙합쪽을 좋아해서 그 쪽으로 추천해줬으면 좋겠습니다.']

In [255]:
music9=get_music('training201806120703.pkl',new_text,7,use_story=False,rate=False)

pos done
cosine done
complete find top7 text and songids
music....loading....
get tag done
cosine done


In [256]:
music9

,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle,similarity,singer,songid,songtitle
0,0.841099,이적,7921208,걱정말아요 그대,0.801412,Chon,5643794,Can`t Wait,0.713090,강세일,532172,사랑은 또 올테니,0.543931,디어,5579211,Forget You
1,0.682792,하현우,8050532,걱정말아요 그대,0.801412,SOMA,30705601,Someday,0.708658,김보경,4277805,그때로 가고싶다,0.494879,소녀시대,2015662,힘 내!
2,0.674278,들국화,4350118,걱정말아요 그대,0.801412,Daniel Caesar,30939859,Get You,0.698534,제이,1762063,Kiss me,0.493506,새봄,30656620,너로 짙은 밤
3,0.630830,일락,3016959,하나만,0.801412,LambC,30731066,Lady,0.698054,JYJ,3550113,소년의 편지,0.492500,토이,1771021,오늘 서울은 하루종일 맑음
4,0.621057,노블레스,1793650,후회는 없어,0.801412,JB,30692018,U & I,0.690179,FTISLAND,2226049,바래,0.486097,빨간의자,5662831,잊혀지는 것
5,0.605200,포맨,2994362,후회한다,0.801412,새소년,30481405,긴 꿈,0.685456,김민종,68743,귀천도애,0.478921,에피톤 프로젝트,1934285,나는 그 사람이 아프다
6,0.585256,소냐,932552,눈물이나,0.759600,피콕,8094430,니가 여자로 보여,0.682862,베이지,4711125,마주보다,0.477827,로맨틱 펀치,2226250,이 밤이 지나면
7,0.580176,리사,1205566,인연,0.751720,Wynn,30698198,LikeMeLoveMe,0.674581,DK,30736562,별이될게,0.475185,노르웨이 숲,7897024,서로의 이별
8,0.579287,The One,1142056,I Do,0.751221,노르웨이 숲,5645712,봄에게,0.672132,포맨,4108716,Angel,0.473605,정은지,8148487,하늘바라기
9,0.578374,박경,30207811,잔상,0.746264,한소아,3469873,고백,0.671673,흘러,2304738,아마도 별 U,0.467714,선우정아,4073182,비온다
